In [16]:
### replicate results of EVmutation with the PABP_YEAST dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model import CouplingsModel
import tools
import scipy
from pathlib import Path
from collections import OrderedDict
# biopython SeqIO
from Bio import SeqIO
from sklearn.metrics import roc_auc_score
import scipy
from collections import OrderedDict
from sklearn.svm import OneClassSVM
from datetime import datetime


In [7]:
ALPHABET_PROTEIN = '-ACDEFGHIKLMNPQRSTVWY'
np.random.seed(0)

In [8]:
# helper functions
def encode(seqs, alphabet=ALPHABET_PROTEIN):
    '''
    Go from letters to numbers
    '''
    aa_to_i = OrderedDict((aa, i) for i, aa in enumerate( alphabet ))
    X = np.asarray([[aa_to_i[x] for x in seq] 
                    for seq in seqs])
    return X, aa_to_i
def one_hot_encode(s):
    return np.vstack([np.zeros(20), np.eye(20)])[s].flatten()

def check_sequence(s, alphabet=ALPHABET_PROTEIN):
    for aa in s:
        if aa not in ALPHABET_PROTEIN:
            return False
    return True
def process_msa_sequence(msa_sequences):
    ''' takes in list of sequences and one-hot encodes the sequences'''
    pos_upper = [x for x in range(len(msa_sequences[0])) if not msa_sequences[0][x].islower()]
    msa_sequences = np.asarray([np.asarray(list(s))[pos_upper] for s in msa_sequences if not 'x' in s])
    msa_sequences = np.asarray([s for s in msa_sequences if check_sequence(s) and len(s)==82])
    msa_sequences = np.asarray(msa_sequences)

    seqs_enc, aa_to_i = encode(msa_sequences)
    oh_enc_seq = []
    for s in seqs_enc:
        oh_enc_seq.append(one_hot_encode(s))
    oh_enc_seq = np.asarray(oh_enc_seq)
    return oh_enc_seq

def valid_weights_from_model(c):
    ### returns only valid weights
    _w = c.weights
    _w_valid = []
    for i in range(c.weights.shape[0]):
        if _w[i] == 0: 
            continue
        _w_valid.append(1/_w[i])
    return _w_valid

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

Preprocessing sequence data

In [9]:
### get all available msa sequences
yeast_seq_str = []
fasta_sequences = SeqIO.parse(open("PABP_YEAST/data/PABP_YEAST.a2m"),'fasta')
for fasta in fasta_sequences:
    yeast_seq_str.append(str(fasta.seq))

processed = process_msa_sequence(yeast_seq_str)
c = CouplingsModel(f"PABP_YEAST/model/PABP_YEAST.model_params")
weights = valid_weights_from_model(c)
assert len(weights) == len(processed)
wildtype_processed, wildtype_weights = processed[0], weights[0]
processed, weights = np.asarray(processed[1:]), np.asarray(weights[1:])
processed, weights = unison_shuffled_copies(processed, weights)

In [5]:
# using the trained svm model on DMS data
wildtype = yeast_seq_str[0]
data = pd.read_csv(
    "PABP_YEAST/data/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)
mutant, label = data['mutant'].to_numpy(), data['linear'].to_numpy()
for i in range(label.shape[0]):
    label[i] = 1 if label[i] > 0.7 else 0
mutant_data = []
for m in mutant:
    original_aa, loc, mutant_aa = m[0], int(m[1:4])-115, m[4]
    assert wildtype[loc] == original_aa
    mutant_data.append(wildtype[:loc]+mutant_aa+wildtype[loc+1:])
mutant_data = np.asarray(mutant_data)
mutant_data = process_msa_sequence(mutant_data)

mutant_data, label = np.asarray(mutant_data), np.asarray(label)
mutant_data, label = unison_shuffled_copies(mutant_data, label)
print(mutant_data.shape)

(1188, 1640)


Experiment 1: Train a OneClassSVM with polynomial degree 2 kernel on MSA sequences. 

In [ ]:
### The classifier has some effect on MSA data. ~2700 out of 5000 total positive samples have been classified correctly from the MSA dataset

clf = OneClassSVM(kernel='poly', degree=2, nu=0.3)
train = processed[:50000]
clf.fit(train, sample_weight=weights[:50000])
pred = clf.predict(processed[50000:55000])
print(pred[np.where(pred==1)].shape)

In [43]:
### The classifier does shitty job for mutant sequences (DMS).
mutant_pred = clf.predict(mutant_data)
print(pred[np.where(mutant_pred==1)].shape)

NameError: name 'clf' is not defined

Experiment 2: Train a OneClassSVM with polynomial degree 2 kernel on DMS sequences.

In [44]:
### also does a shitty job. Probably because too many features and did not shuffle sequences.
import sklearn
clf2 = sklearn.svm.SVC(kernel='poly', degree=2)
split = mutant_data.shape[0]//5 * 4
mut_train = mutant_data[:split]
mut_test = mutant_data[split:]
clf2.fit(mut_train, label[:split])

NameError: name 'pred' is not defined

Experiement 3: Gaussian Kernel OneClassSVM

In [10]:
clf = OneClassSVM(kernel='rbf', nu=0.3)
train = processed[:50000]
clf.fit(train, sample_weight=weights[:50000])
pred = clf.predict(processed[50000:55000])
print(pred[np.where(pred==1)].shape)

(3552,)


In [12]:
### The classifier does shitty job for mutant sequences (DMS).
mutant_pred = clf.predict(mutant_data)
print(pred[np.where(mutant_pred==1)].shape)

(1188,)


Experiement 4: explicitly model pairwise features. the original d features become d^2 features. SVM still uses poly deg-2 kernel

In [14]:
d = processed.shape[1]
num_seq = processed.shape[0]
rich_msa = np.zeros((num_seq//10, d**2))

# scipy.sparse.csr_matrix((num_seq, d**2))

In [10]:
# explicitly model pairwise interactions as features. This cell takes more than 1 hour to run
for seq in range(num_seq//10):
    for i in range(d):
        for j in range(i, d):
            if processed[seq][i] == 1 and processed[seq][j] == 1 or i == j:
                rich_msa[seq][i*d + j] = 1
rich_msa = scipy.sparse.csr_matrix(rich_msa, (num_seq//10, d**2))

test = np.zeros((mutant_data.shape[0], d**2))
for seq in range(mutant_data.shape[0]):
    for i in range(d):
        for j in range(i, d):
            if mutant_data[seq][i] == 1 and mutant_data[seq][j] == 1 or i == j:
                test[seq][i*d + j] = 1

test_sparse = scipy.sparse.csr_matrix(test, (mutant_data.shape[0], d**2))


In [15]:
import pickle

# with open("store_rich_msa.obj", "wb") as file:
#     pickle.dump(rich_msa, file)

# with open("store_rich_test.obj", "wb") as file:
#     pickle.dump(test_sparse, file)

# with open("store_rich_weights.obj", "wb") as file:
#     pickle.dump(weights, file)

# with open("store_test_label.obj", "wb") as file:
#     pickle.dump(label, file)

rich_msa, test_sparse = None, None
label, weights = None, None
with open("store_rich_msa.obj", "rb") as file:
    rich_msa = pickle.load(file)

with open("store_rich_test.obj", "rb") as file:
    test_sparse = pickle.load(file)

with open("store_rich_weights.obj", "rb") as file:
    weights = pickle.load(file)

with open("store_test_label.obj", "rb") as file:
    label = pickle.load(file)
    
assert rich_msa.shape[1] == test_sparse.shape[1]
assert label.shape[0] == test_sparse.shape[0]

# ### expand wildtype
wt = [0 for _ in range(d**2)]
for i in range(d):
    for j in range(i, d):
        if wildtype_processed[i] == 1 and wildtype_processed[j] == 1 or i == j:
            wt[i*d + j] = 1

In [11]:
# for n in [0.1, 0.15, 0.2, 0.25, 0.3]:
#     clf = OneClassSVM(kernel='linear', nu=n)
#     clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
#     pred = clf.predict(test_sparse)
#     cnt = 0 
#     for i in range(mutant_data.shape[0]):
#         if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
#             cnt += 1
#     print(f'kernel: linear, nu: {n}, result: ', cnt)
#     print('wt: ', clf.predict([wt]))

# ### prediction of wildtype
# wt = [0 for _ in range(d**2)]
# for i in range(d):
#     for j in range(i, d):
#         if wildtype_processed[i] == 1 and wildtype_processed[j] == 1 or i == j:
#             wt[i*d + j] = 1
# clf.predict([wt])

Experiment 4.1: Hyperparameter tuning

In [ ]:
for n in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    for deg in [1, 2, 3]:
        clf = OneClassSVM(kernel='poly', degree = deg, nu=n)
        clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
        pred = clf.predict(test_sparse)
        cnt = 0 
        for i in range(mutant_data.shape[0]):
            if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
                cnt += 1
        print(f'kernel: poly, degree: {deg}, nu: {n}, result: ', cnt)
        print(pred[:20])
        print("wild type pred: ", clf.predict([wt]))
    
    clf = OneClassSVM(kernel='rbf', nu=n)
    clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
    pred = clf.predict(test_sparse)
    cnt = 0 
    for i in range(mutant_data.shape[0]):
        if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
            cnt += 1
    
    print(f'kernel: rbf, nu: {n}, result: ', cnt)
    print(pred[:20])
    print("wild type pred: ", clf.predict([wt]))

    clf = OneClassSVM(kernel='sigmoid', nu=n)
    clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
    pred = clf.predict(test_sparse)
    cnt = 0 
    for i in range(mutant_data.shape[0]):
        if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
            cnt += 1
    print(f'kernel: rbf, nu: {n}, result: ', cnt)
    print(pred[:20])
    print("wild type pred: ", clf.predict([wt]))


Results:
    Only (kernel = 'rbf', nu = 0.2) and (kernel = 'rbf', nu = 0.3) produced results in which the prediction of mutant sequences were not all 1's or all -1's. For nu = 0.2, 716 sequences were classified correctly. For nu = 0.3, 543 sequences were classified correctly. There are a total of 673 (pos) + 515 (neg) = 1188 sequences. 

Experiement 4.2: Look into the weights assigned to each feature in the linear model, see if SVM has learned anything interesting about pairwise weights. Compare with the Potts model.

In [21]:
NAME="PABP_YEAST"
# load parameters from file to create a pairwise model
c = CouplingsModel(f"PABP_YEAST/model/{NAME}.model_params")

potts_params = c.J_ij
print(potts_params.shape)

potts_weights_arr = np.zeros(2689600)
for i in range(82):
    for aa1 in range(20):
        for j in range(i, 82):
            for aa2 in range(20):
                potts_weights_arr[(i*20 + aa1) * d + j*20 + aa2] = c.J_ij[i][j][aa1][aa2]


(82, 82, 20, 20)


In [22]:
# train a linear model, which should theoretically be similar to the potts model.

with open('result.txt', 'a') as file:
    date_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    file.write(date_time)
    file.write('\n\n')
    for n in [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5]:
        clf = OneClassSVM(kernel='linear', nu=n)
        clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
        pred = clf.predict(test_sparse)
        cnt = 0 
        for i in range(mutant_data.shape[0]):
            if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
                cnt += 1
        print(f'kernel: linear, nu: {n}, result: ', cnt)
        file.write(f'kernel: linear, nu: {n}, result: {cnt}\n')
        print(pred[:20])
        file.write('\n')
        print(np.sum(pred))
        file.write(f'Sum: {np.sum(pred)}\n')
        print("wild type pred: ", clf.predict([wt]))
        file.write(f'wild type pred : {clf.predict([wt])}\n')
        w = clf.coef_.toarray()
        w = w.flatten()

        sprm = scipy.stats.spearmanr(w, potts_weights_arr)
        print(f'spearman for nu = {n} is: {sprm}')
        file.write(f'spearman for nu = {n} is: {sprm}\n\n\n')

kernel: linear, nu: 0.001, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.001 is: SpearmanrResult(correlation=0.0799076641387388, pvalue=0.0)
kernel: linear, nu: 0.002, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.002 is: SpearmanrResult(correlation=0.07981870911144529, pvalue=0.0)
kernel: linear, nu: 0.01, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.01 is: SpearmanrResult(correlation=0.08089272402400162, pvalue=0.0)
kernel: linear, nu: 0.02, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.02 is: SpearmanrResult(correlation=0.08254206570403295, pvalue=0.0)
kernel: linear, nu: 0.1, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.1 is: SpearmanrResult(correlation=0.08719157913545321, pvalue=0.0)
kernel: linear, nu: 0.2,

In [23]:
with open('result.txt', 'a') as file:
    date_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    file.write(date_time)
    file.write('\n\n')
    # train a rbf model, which should theoretically be similar to the potts model.
    for n in [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.201, 0.202, 0.21, 0.22, 0.28, 0.3, 0.5]:
        clf = OneClassSVM(kernel='rbf', nu=n)
        clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
        pred = clf.predict(test_sparse)
        cnt = 0 
        for i in range(mutant_data.shape[0]):
            if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
                cnt += 1

        print(f'kernel: rbf, nu: {n}, result: ', cnt)
        file.write(f'kernel: rbf, nu: {n}, result: {cnt}\n')
        print(pred[:20])
        print(np.sum(pred))
        file.write(f'Sum: {np.sum(pred)}\n')
        print("wild type pred: ", clf.predict([wt]))
        file.write(f'wild type pred : {clf.predict([wt])}\n')
        sv = clf.support_vectors_
        c = clf.dual_coef_
        w = (c @ sv).toarray().flatten()

        sprm = scipy.stats.spearmanr(w, potts_weights_arr)
        print(f'spearman for nu = {n} is: {sprm}')
        file.write(f'spearman for nu = {n} is: {sprm}\n\n\n')


kernel: rbf, nu: 0.001, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.001 is: SpearmanrResult(correlation=0.08430774677159406, pvalue=0.0)
kernel: rbf, nu: 0.002, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.002 is: SpearmanrResult(correlation=0.08442035815768388, pvalue=0.0)
kernel: rbf, nu: 0.01, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.01 is: SpearmanrResult(correlation=0.08379710418208093, pvalue=0.0)
kernel: rbf, nu: 0.02, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.02 is: SpearmanrResult(correlation=0.08362051515625542, pvalue=0.0)
kernel: rbf, nu: 0.1, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.1 is: SpearmanrResult(correlation=0.08456227564812689, pvalue=0.0)
kernel: rbf, nu: 0.2, result:  691
[ 1

In [24]:
with open('result.txt', 'a') as file:
    date_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    file.write(date_time)
    file.write('\n\n')
    # train a rbf model, which should theoretically be similar to the potts model.
    for n in [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.201, 0.202, 0.21, 0.22, 0.28, 0.3, 0.5]:
        clf = OneClassSVM(kernel='poly', degree=2, nu=n)
        clf.fit(rich_msa, sample_weight=weights[:rich_msa.shape[0]])
        pred = clf.predict(test_sparse)
        cnt = 0 
        for i in range(mutant_data.shape[0]):
            if (pred[i]== 1 and label[i] == 1) or (pred[i] == -1 and label[i] == 0):
                cnt += 1

        print(f'kernel: poly-2, nu: {n}, result: ', cnt)
        file.write(f'kernel: poly-2, nu: {n}, result: {cnt}\n')
        print(pred[:20])
        print(np.sum(pred))
        file.write(f'Sum: {np.sum(pred)}\n')
        print("wild type pred: ", clf.predict([wt]))
        file.write(f'wild type pred : {clf.predict([wt])}\n')
        sv = clf.support_vectors_
        c = clf.dual_coef_
        w = (c @ sv).toarray().flatten()

        sprm = scipy.stats.spearmanr(w, potts_weights_arr)
        print(f'spearman for nu = {n} is: {sprm}')
        file.write(f'spearman for nu = {n} is: {sprm}\n\n\n')


kernel: poly-2, nu: 0.001, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.001 is: SpearmanrResult(correlation=0.08426271678544617, pvalue=0.0)
kernel: poly-2, nu: 0.002, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.002 is: SpearmanrResult(correlation=0.08478842538701295, pvalue=0.0)
kernel: poly-2, nu: 0.01, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.01 is: SpearmanrResult(correlation=0.08578413116966253, pvalue=0.0)
kernel: poly-2, nu: 0.02, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.02 is: SpearmanrResult(correlation=0.08616116699946114, pvalue=0.0)
kernel: poly-2, nu: 0.1, result:  673
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1188
wild type pred:  [1]
spearman for nu = 0.1 is: SpearmanrResult(correlation=0.09002485120390699, pvalue=0.0)
kernel: poly-2, nu: 0.2

In [30]:
print(sv.shape)
print(c.shape)

(472, 2689600)
(1, 472)


In [31]:
w.shape

(2689600,)

In [32]:
# w = clf.coef_.toarray()
# w = w.flatten()

sprm = scipy.stats.spearmanr(w, potts_weights_arr)
print(f'spearman for nu = {n} is: {sprm}')

spearman for nu = 0.001 is: SpearmanrResult(correlation=0.08430774677159406, pvalue=0.0)
